# MiniCPM-V 2.6

In [1]:
! python -m pip install --upgrade pip
! pip install -r MiniCPM_requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 31.5 MB/s eta 0:00:00 0:00:01
  Attempting uninstall: pip
    Found existing installation: pip 23.3.1
    Uninstalling pip-23.3.1:
      Successfully uninstalled pip-23.3.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 110.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 143.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.2/670.2 MB 145.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 342.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 159.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 210.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 323.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 301.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 183.6 MB/s eta 0:00:

In [8]:
import os
import torch
from PIL import Image
from transformers import AutoModel, AutoTokenizer
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score, classification_report
from tqdm import tqdm
import json

# Open log file
log_path = './results/minicpm/'
os.makedirs(log_path, exist_ok=True)
log_file = os.path.join(log_path, 'prediction_log.txt')
log_file = open(log_file, 'w')

# Load data
df_table_prd = pd.read_csv('../LMM_sewerML/results/df_table_prd.csv')
df_table_dsc = pd.read_csv('../LMM_sewerML/results/df_table_dsc.csv')
image_dir = '../LMM_sewerML/images'

# Load MiniCPM model and tokenizer
model = AutoModel.from_pretrained('openbmb/MiniCPM-V-2_6', trust_remote_code=True, 
                                  attn_implementation='sdpa', torch_dtype=torch.bfloat16)
model = model.eval().cuda()
tokenizer = AutoTokenizer.from_pretrained('openbmb/MiniCPM-V-2_6', trust_remote_code=True)

# Define the prompt template
prompt_template = """
You are a virtual sewer technician with the capability to analyze images from CCTV cameras taken inside sewer pipes.
Your task is to examine each image and provide a concise, yet accurate, summary for retrieval. 
After summarizing, you must classify the image as defect types.
While providing the summary, remember the following guidelines:"
1) Provide a general overview of the image that you see, describing important elements such image clarity, lighting conditions, type of pipe (concrete, PVC, ...), presence of water.
2) Check for defects in the sewer pipes in the image.
3) Pipes in good condition usually show a smooth, unbroken surface, no visible signs of damage like cracks or collapses, and an absence of blockages such as roots.
4) On the other hand, you can have the following defects:
4a) Cracks, Breaks, and Collapses (RB): Identify visible cracks along the pipe, instances where the pipe has fractured or completely broken apart, and areas where the pipe has collapsed.
This includes longitudinal cracks, circumferential breaks, and complete structural failures that compromise the integrity of the sewer system.
4b) Surface Damage (OB): Detect areas of the pipe's interior that exhibit signs of wear, erosion, or damage on the surface.
This includes minor scratches, pitting, scaling, or any form of deterioration that affects the pipe's surface but does not necessarily penetrate deeply into the structure.
4c) Production Error (PF): Identify defects that originated during the pipe's manufacturing process, such as inconsistent pipe thickness, improper joint alignment, or material imperfections.
These are flaws that were introduced before installation and could potentially affect the pipe's performance or longevity.
4d) Deformations (DE): Recognize any alterations in the shape of the pipe, such as bending, sagging, or bulging, that indicate a deformation.
This includes both minor deformations that may affect flow efficiency and major deformations that threaten the pipe's structural integrity.
4e) Roots (RO): Detect the presence of roots infiltrating the sewer pipe, whether through joints, cracks, or other vulnerabilities.
This involves identifying both the initial stages of root intrusion and the more advanced stages where roots have significantly obstructed the pipe.
5) Additional considerations while analyzing the images: do not consider blurred text or user-defined circled areas in the images.
6) You will always try to describe the image that you see. Provide the output in JSON format as follows:

{ "DESCRIPTION": "<Description of the image that you see>", "CODE": "<Defect Code>"}

Note: The "CODE" can be selected from "RB", "OB", "PF", "DE", "RO". If no defect is detected, set "CODE" to "NoDefect".
"""

# Store predictions and actual labels
predictions = []
actual_labels = []
descriptions = []

# Add tqdm progress bar
for idx, row in tqdm(df_table_prd.iterrows(), total=len(df_table_prd), desc="Processing images"):
    img_id = row['img_id']
    ground_truth = row['defect_type']
    img_path = os.path.join(image_dir, img_id)
    
    # Check if image file exists
    if not os.path.exists(img_path):
        log_file.write(f"Image {img_id} not found. Skipping.\n")
        continue

    # Load image
    image = Image.open(img_path).convert('RGB')
    
    # Prepare the messages for MiniCPM
    question = prompt_template
    msgs = [{'role': 'user', 'content': [image, question]}]
    
    # Generate response
    try:
        res = model.chat(image=None, msgs=msgs, tokenizer=tokenizer)
        # Extract defect prediction and description from model output
        response_json = json.loads(res)
        description = response_json.get("DESCRIPTION", "No description provided")
        predicted_defect = response_json.get("CODE", "NoDefect")
    except json.JSONDecodeError:
        # Handle invalid JSON
        log_file.write(f"Invalid JSON response for Image ID {img_id}: {res}\n")
        description = "Error in JSON response"
        predicted_defect = "Error"

    # Append results to lists
    predictions.append(predicted_defect)
    actual_labels.append(ground_truth)
    descriptions.append(description)

    # Log intermediate outputs
    log_file.write(f"Image ID: {img_id} | Predicted: {predicted_defect} | Ground Truth: {ground_truth}\n")

# Calculate evaluation metrics
accuracy = accuracy_score(actual_labels, predictions)
f1 = f1_score(actual_labels, predictions, average='weighted')
report = classification_report(actual_labels, predictions)

log_file.write(f"\nAccuracy: {accuracy:.4f}\n")
log_file.write(f"F1 Score: {f1:.4f}\n")
log_file.write("\nClassification Report:\n")
log_file.write(report)

# Close log file
log_file.close()

# Save results to CSV files
df_table_prd['minicpm-v 2.6'] = predictions
df_table_prd.to_csv(log_path + 'df_table_prd.csv', index=False)

df_table_dsc['minicpm-v 2.6'] = descriptions
df_table_dsc.to_csv(log_path + 'df_table_dsc.csv', index=False)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/1.36k [00:00<?, ?B/s]

configuration_minicpm.py:   0%|          | 0.00/3.28k [00:00<?, ?B/s]

modeling_navit_siglip.py:   0%|          | 0.00/41.8k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/openbmb/MiniCPM-V-2_6:
- modeling_navit_siglip.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/openbmb/MiniCPM-V-2_6:
- configuration_minicpm.py
- modeling_navit_siglip.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

model-00001-of-00004.safetensors:   0%|          | 0.00/4.87G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/4.93G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:653: UserWarning: Not enough free disk space to download the file. The expected file size is: 4330.87 MB. The target location /root/.cache/huggingface/hub/models--openbmb--MiniCPM-V-2_6/blobs only has 3206.57 MB free disk space.
  warnings.warn(


model-00003-of-00004.safetensors:   0%|          | 0.00/4.33G [00:00<?, ?B/s]

OSError: [Errno 28] No space left on device